In [2]:
# Cesarine Graham
# AST 5765 Homework 5
# 09/20/2025

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import astropy.io.fits as fits 

### 2. (10 Points) Here you will continue the sigma-clipping we worked on during Practicum3. Repeat the steps from problem 2b) on the subsample you created in 2b), to make a sub-subsample.

### Print the new mean, median, and standard deviation. How different are the final mean and median?

### How close is the final standard deviation to that expected for the Poisson distribution? Will this method always remove every bad pixel?

In [7]:
# documentation sources:
# https://numpy.org/doc/2.2/reference/random/generated/numpy.random.poisson.html
# https://numpy.org/doc/2.2/reference/random/generated/numpy.random.uniform.html

# number of photons
N = 10000

# creating our initial array
data_ccd = np.zeros(400)
# random.poisson(lam=1.0, size=None)
data_ccd[ :396] = np.random.poisson(N, 396)  #396 draws from a poisson distribution for 10,000 photons

N1 = 0
N2 = 1e6
# random.uniform(low=0.0, high=1.0, size=None)
data_ccd [396: ]= np.random.uniform( N1, N2, 4)

#print(data_ccd)

In [9]:
print("Raw data mean: " , np.mean(data_ccd) , "Raw data median: " , np.median(data_ccd) )

sigma = np.std(data_ccd)
med_ccd = np.median(data_ccd)

low_range = med_ccd - 5*sigma
high_range = med_ccd =+ 5*sigma

filtered_data_ccd = (data_ccd[(low_range < data_ccd) & (data_ccd < high_range)] )

print("Filtered mean: " , np.mean(filtered_data_ccd) , "Filtered median: " , np.median(filtered_data_ccd) )

Raw data mean:  12665.041588871427 Raw data median:  9996.0
29578.980798159206
Filtered mean:  10181.694431905478 Filtered median:  9996.0


#### The median is lower than the mean due to the outliers in the raw dataset! After applying the mask (filter), the medians are still the same, but the mean is different and much closer to the value of the median.

In [12]:
print(np.std(data_ccd))
print(np.std(filtered_data_ccd))
print(np.sqrt(np.mean(filtered_data_ccd)))

29578.980798159206
3610.0963968934984
100.9043826199114


#### In the Poisson Distribution, the standard deviation is equal to the square root of its mean. The standard deviation of our filtered dataset was calculated to be 3610.1, but the square of the mean is 100.9. The standard deviation of the unfiltered dataset was calculated to be roughly, 29579 which is VERY far off from the proper square root of the mean. 

#### Therefore, performing just masking the dataset with some sigma will not and has not removed every bad pixel.?